related_artists.json을 활용해 데이터 증강을 진행
1. related_artists.json을 불러옴
2. 해당 json을 root artist를 포함한 문장들로 만듬
3. 배열별로 각각 artist의 장르를 가져와서 새로운 문장들을 만듬
4. 중복 제거하고 장르의 count순으로 정렬
5. 중앙 정렬해서 최종 문장으로 만듬

In [ ]:
import argparse
import logging
import json
from tqdm import tqdm
import time
import random
import math
import pickle

In [ ]:
file_path = "./dataset/spotify_dataset.json"

with open(file_path, 'r') as file:
    origin_data = json.load(file)

In [ ]:
file_path = "./dataset/related_artists.json"

with open(file_path, 'r') as file:
    related_data = json.load(file)

In [ ]:
file_path = "./dataset/genre_artists.json"

with open(file_path, 'r') as file:
    genre_artists_json = json.load(file)

In [ ]:
def center_align(arr):
    aligned_arr = ['0' for i in range(len(arr))]
    center = int((len(arr))/2)
    
    for i in range(len(arr)):
        aligned_arr[center + ((-1)**i) * math.ceil(i/2)] = arr[i]
    
    return aligned_arr

In [ ]:
def get_artist_genre(artist_id, origin_data):
    for d in origin_data:
        if d['artist_id'] == artist_id:
            return d['genres']

In [ ]:
# 장르가 들어있는 1차원 배열을 입력하면 카운트 순으로 중앙 정렬
def get_center_aligned_list(genres, genre_artists_json):
    tmp_arr = []
    
    for g in genres:
        for d in genre_artists_json:
            if g == d['genre']:
                tmp_arr.append([g, d['count']])
                break
    
    tmp_arr.sort(key=lambda x:-x[1])
    tmp_sorted_arr = []
    for t in tmp_arr:
        tmp_sorted_arr.append(t[0])
    
    return center_align(tmp_sorted_arr)

In [ ]:
genre_sentences = []

for idx, d in enumerate(tqdm(related_data)):
    tmp_sentence = get_artist_genre(d['artist_id'], origin_data)
    
    for artist in d['related_artists']:
        tmp_sentence = tmp_sentence + get_artist_genre(artist, origin_data)
    
    #중복 제거
    tmp_sentence = list(set(tmp_sentence))
    
    tmp_aligned_list = get_center_aligned_list(tmp_sentence, genre_artists_json)
    genre_sentences.append(tmp_aligned_list)
    

In [ ]:
# genre_sentences를 저장
with open('./dataset/augmented_genre_sentences.pkl','wb') as f:
    pickle.dump(genre_sentences, f)